In [ ]:
pip install lsfb_dataset

In [11]:
from lsfb_dataset import Downloader

downloader = Downloader(
    dataset='isol',
    destination="../Database/Dataset",
    signers=list(range(0, 5)),
    include_cleaned_poses=True,
    include_raw_poses=False,
    include_videos=False,
    landmarks=['pose', 'left_hand', 'right_hand'],
    skip_existing_files=True,
    n_labels=10,
)

downloader.download()


TypeError: Downloader.__init__() got an unexpected keyword argument 'n_labels'

In [ ]:
import lsfb_dataset
from lsfb_dataset import LSFBIsolConfig, LSFBIsolLandmarks

dataset = LSFBIsolLandmarks(LSFBIsolConfig(
    root="../Database/Dataset",
))
features, target = dataset[0]
